In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

## helper functions

In [7]:
def get_url(book_title, book_author):
    prefix = 'https://www.goodreads.com/search?utf8=%E2%9C%93&q='
    sufix = '&search_type=books'
    query = ''
    
    if '(' in book_title:
        book_title_s = book_title.split('(')[0]
    elif '[' in book_title:
        book_title_s = book_title.split('[')[0]
    elif':' in book_title:
        book_title_s = book_title.split(':')[0]
    else:
        book_title_s = book_title
    
    for word in book_author.strip().split(' '):
        query+=word + '+'
    
    for word in book_title_s.strip().split(' '):
        query+=word + '+'
       
    return prefix + query[:-1] + sufix
        

## Get list of kindle books from romancebookworms

In [2]:
url = "https://www.romancebookworms.com/kindle/"
results = requests.get(url)
soup = BeautifulSoup(results.text, "html.parser")
book_div = soup.find_all('div', class_='sqs-block-content')

In [3]:
books = []
for i in range(4,50, 3):
    if len(book_div[i-1].find_all('h2')) > 1:
        category = book_div[i-1].find_all('h2')[1]
        category = str(category).replace('<h2 style="white-space:pre-wrap;">', '')
        category = str(category).replace('</h2>', '')
    else:
        category = book_div[i-1].h2.text
    
    titles = book_div[i].find_all('b')
    links = book_div[i].find_all('a')
    keywords = book_div[i].find_all('br')
    last_keyword_index = 1
    for j in range(len(titles)):
        book_title = titles[j].text
        book_title = str(book_title).strip()
        link = links[j]['href']
        author = titles[j].nextSibling
        author = str(author).replace('by ', '').strip()
        book_keywords = keywords[last_keyword_index].nextSibling
        book_keywords = str(book_keywords).replace('Keywords: ', '').strip()
        last_keyword_index += 3 
        books.append({'category':category,'book_title': book_title, 'link': link, 'author':author, 'keywords':book_keywords})
df = pd.DataFrame.from_dict(books)

## Get Good Reads data

In [9]:
df['goodreads_url'] = df.apply(lambda x: get_url(x.book_title, x.author), axis=1)

In [59]:
url_goodreads = 'https://www.goodreads.com/search?utf8=%E2%9C%93&q=S.+Breaker+Save+Yourself:+A+Portal+Fantasy+Sci-fi+Adventure&search_type=books'
results = requests.get(url_goodreads)
soup = BeautifulSoup(results.text, "html.parser")

In [74]:
def get_avg_rating_num_rating(url_goodreads):
    print(url_goodreads)
    results = requests.get(url_goodreads)
    parsed_html = BeautifulSoup(results.text, "html.parser")
    avg_rating = 0
    num_ratings = 0
    if parsed_html.find('h3', class_="searchSubNavContainer"):
        if parsed_html.find('h3', class_="searchSubNavContainer").text != 'No results.':
            ratings = parsed_html.find_all('span', class_ ="minirating")[0].text.strip().split(' — ')
            #it was amazing - really liked it 
            avg_rating = ratings[0].replace(' avg rating', '')
            avg_rating = ratings[0].replace('it was amazing ', '')
            avg_rating = ratings[0].replace('really liked it ', '')
            avg_rating = ratings[0].replace('did not like it ', '')
            avg_rating = ratings[0].replace('liked it ', '')
            
            num_ratings = ratings[1].replace(' ratings','').replace(',','')
            num_ratings = ratings[1].replace(' rating','').replace(',','')
            print(avg_rating)
            print(num_ratings)
    return avg_rating, num_ratings

In [75]:
df[['avg_rating', 'num_ratings']] = df.apply(lambda x: get_avg_rating_num_rating(x['goodreads_url']), axis=1, result_type='expand')

https://www.goodreads.com/search?utf8=%E2%9C%93&q=S.+Breaker+Save+Yourself:+A+Portal+Fantasy+Sci-fi+Adventure&search_type=books
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Aisha+Urooj+Midnight+Prince&search_type=books
3.03
74
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Angelina+J.+Steffort+White&search_type=books
3.74
1598
https://www.goodreads.com/search?utf8=%E2%9C%93&q=L.+S.+O'Dea+Escape&search_type=books
4.03
690
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Magan+Vernon+Defy+The+Stars&search_type=books
4.15
41
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Rebecca+Rode+Numbers+Game&search_type=books
3.97
1683
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Jessa+Russo+Girls+Who+Lie+Together&search_type=books
3.43
176
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Casey+L.+Bond+Gravebriar&search_type=books
4.17
612
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Tricia+Copeland+Kingdom+of+Embers&search_type=books
4.18
400
https://www.goodreads.com/search?u

4.27
199
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Aubrey+Elle+A+Spot+of+Earl+Slay&search_type=books
4.59
51
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Sadie+Bosque+A+Deal+with+the+Earl&search_type=books
4.17
1940
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Tanya+Anne+Crosby+The+King's+Favorite&search_type=books
4.19
1618
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Amanda+Mariel+Her+Perfect+Rogue&search_type=books
3.72
167
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Merry+Farmer+His+Perfect+Bride&search_type=books
3.99
2002
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Golden+Angel+A+Season+for+Treason&search_type=books
4.27
627
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Margery+Scott+Cade&search_type=books
3.96
723
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Parker+J.+Cole+A+Match+for+Bernadette&search_type=books
4.62
104
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Kelly+McClymer+The+Fairy+Tale+Bride&search_type=books
3.47
4716
h

4.04
1816
https://www.goodreads.com/search?utf8=%E2%9C%93&q=MK+Robert+The+Shield+and+Axe&search_type=books
4.32
121
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Kathleen+Ball+Teagan:+Cowboy+Strong&search_type=books
4.38
1016
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Regina+Kammer+Disputed+Boundaries&search_type=books
4.50
6
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Jenna+Jaxon+A+Countess+of+Convenience&search_type=books
3.84
25
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Cat+Cahill+Forever+Christmas&search_type=books
4.48
359
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Daria+Vernon+The+Rogue's+Last+Letter&search_type=books
4.30
33
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Charlotte+Jardine+His+Lady+of+Fire:+a+heart-warming+sweet+Victorian+Romance&search_type=books
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Brianne+Gillen+Difficult&search_type=books
4.07
30
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Stephanie+Flynn+Pirate's+Prize:+A

3.59
152
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Rose+Sinclair+The+8th+Rank&search_type=books
3.33
51
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Terri+Brisbin+Rising+Fire&search_type=books
3.73
138
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Cassia+Briar+Captive+Beauty&search_type=books
4.07
350
https://www.goodreads.com/search?utf8=%E2%9C%93&q=P.S.+Scott+The+Devil's+Curse&search_type=books
4.32
22
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Eileen+Mueller+Ezaara&search_type=books
4.31
475
https://www.goodreads.com/search?utf8=%E2%9C%93&q=M.+Lynn+Golden+Curse:+A+Fantasy+Romance+Novel&search_type=books
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Lisette+Marshall+Velvet&search_type=books
3.56
622
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Briar+Boleyn+Queen+of+Roses&search_type=books
3.94
429
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Karen+Kincy+Demonic+Prince&search_type=books
3.74
247
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Je

3.80
222
https://www.goodreads.com/search?utf8=%E2%9C%93&q=A.+L.+Tippett+A+Dragon's+Mind&search_type=books
really liked it 4.00
68
https://www.goodreads.com/search?utf8=%E2%9C%93&q=TJ+Nichols+Vanity+and+other+Monsters&search_type=books
4.52
130
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Melissa+A.+Craven+Emerge&search_type=books
4.05
952
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Multiple+Authors+Urban+Mythic+Box+Set:+Eleven+Novels+of+Mythic+Adventure+and+Romance&search_type=books
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Maggie+Shaw+Bounty+Hunter&search_type=books
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Sharla+Wylde+Awakened&search_type=books
really liked it 4.00
48
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Shannon+Pemrick+Destiny&search_type=books
3.80
431
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Devon+Vesper+The+Elven+King's+Captive&search_type=books
3.71
266
https://www.goodreads.com/search?utf8=%E2%9C%93&q=N.+Z.+Nasser+Midlife+News&

...

https://www.goodreads.com/search?utf8=%E2%9C%93&q=Aisha+Urooj+Midnight+Prince&search_type=books
3.01
76
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Sadie+Bosque+A+Deal+with+the+Earl&search_type=books
4.17
1940
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Christina+Berry+The+Road+Home&search_type=books
4.33
113
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Golden+Angel+A+Season+for+Treason&search_type=books
4.27
627
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Dee+James+Bound+By+The+Marriage+Contract&search_type=books
3.84
234
https://www.goodreads.com/search?utf8=%E2%9C%93&q=J.A.+Roles+Beyond+The+Shadows:+Shadow+Institute&search_type=books
3.80
60
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Sinistre+Ange+His+Omega+Babygirl&search_type=books
3.72
180
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Shanae+Johnson+The+Brides+of+Purple+Heart+Ranch+Boxset+Volume+1&search_type=books
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Elyse+Kelly+Coming+in+Hot&search

it was amazing 5.00
2
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Tracy+Reed+The+Good+Girl+Part+One&search_type=books
3.66
175
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Lisa+Lang+Blakeney+Claimed&search_type=books
3.90
806
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Coco+Miller+Carmine&search_type=books
3.64
258
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Aarti+V+Raman+Days+of+Our+Lives&search_type=books
3.87
38
https://www.goodreads.com/search?utf8=%E2%9C%93&q=J.L.+Campbell+A+Baker's+Dozen:+13+Steps+to+Distraction&search_type=books
4.50
26
https://www.goodreads.com/search?utf8=%E2%9C%93&q=R.K.+Dreaming+Witch+In+Charm's+Way&search_type=books
4.28
453
https://www.goodreads.com/search?utf8=%E2%9C%93&q=D.D.+Sand+Wolf+Cabin&search_type=books
4.05
78
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Didi+Sand+What+Happens+In+Paradise&search_type=books
4.52
27
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Kyra+Parsi+Fool+Them+Once&search_type=books
4.41
51
https

4.03
3085
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Vinni+George+A+Foot+in+the+Door&search_type=books
4.21
117
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Eliza+Loveless+Taming+the+Minotaur&search_type=books
2.33
9
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Wendy+Rathbone+Little+Boy+Blu&search_type=books
3.88
580
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Ariella+Zoelle+Suntastic+Fun:+The+Complete+Series&search_type=books
3.90
21
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Jenny+L+Black+A+Pack+Extended&search_type=books
4.16
85
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Jocelynn+Drake+Claiming+Marcus&search_type=books
4.01
2216
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Becca+Seymour+Thicker+Than+Water&search_type=books
3.99
268
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Abrianna+Denae+Black+Ice+Heart&search_type=books
4.10
31
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Lissa+Kasey+Night+with+the+Morningstar&search_type=books

4.13
1207
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Sarah+Zane+Beautiful+Little+Fool&search_type=books
4.19
21
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Jayne+Rylon+Kate's+Crew&search_type=books
3.71
5693
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Leighton+Greene+Learn+the+Rules&search_type=books
3.97
411
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Livia+Grant+Black+Light:+Valentine+Roulette&search_type=books
4.42
961
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Misha+Horne+Hurt+Me,+Daddy&search_type=books
4.15
1694
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Blaine+D.+Arden+The+Forester&search_type=books
3.59
194
https://www.goodreads.com/search?utf8=%E2%9C%93&q=B.+Beckett+Amantes&search_type=books
3.75
4
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Jamie+Merrick+Surprising+Their+Secretary&search_type=books
4.12
73
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Piper+Scott+Alpha+Awakened&search_type=books
3.78
1499
https://www.goodreads.

4.32
22
https://www.goodreads.com/search?utf8=%E2%9C%93&q=J.+Haney+Minor++Adjustments&search_type=books
3.67
3
https://www.goodreads.com/search?utf8=%E2%9C%93&q=J.+Haney+&+S.I.+Hayes+Faking+It&search_type=books
3.61
172
https://www.goodreads.com/search?utf8=%E2%9C%93&q=S.I.+Hayes+The+Solitary+Road&search_type=books
4.11
18
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Katie+Vincent+Merysekhmet&search_type=books
4.29
7
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Skye+MacKinnon+Thorrn&search_type=books
4.16
275
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Fil+Reid+Guinevere:+The+Dragon+Ring&search_type=books
4.42
211
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Carlotta+Hughes+Thorn's+Dove&search_type=books
4.13
252
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Kathy+Haan+Fae+Gods&search_type=books
3.57
7
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Emily+Royal+What+the+Hart+Wants&search_type=books
4.20
603
https://www.goodreads.com/search?utf8=%E2%9C%93&q=Pa

In [84]:
df.to_excel('book_recommendations.xlsx')